In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import nsys_reader

 # Compare results on RTX 3050Ti vs RTX 2080 Ti vs A100
 
 
A100_path = "../../../data/decompression/compare/a100/"
RTX2080TI_path = "../../../data/decompression/compare/RTX2080ti/"
RTX3050TI_path = "../../../data/decompression/compare/RTX3050ti/"

GPU_A100 = 'A100'
GPU_RTX2080TI = 'RTX2080TI'
GPU_RTX3050TI = 'RTX3050TI'

FILE_SIZE = 524288000




In [4]:
A100_result_df = nsys_reader.fix_chunks(pd.read_csv(A100_path + '/results.csv'), FILE_SIZE)
RTX2080TI_result_df = nsys_reader.fix_chunks(pd.read_csv(RTX2080TI_path + '/results.csv'), FILE_SIZE)
RTX3050TI_result_df = nsys_reader.fix_chunks(pd.read_csv(RTX3050TI_path + '/results.csv'), FILE_SIZE)

A100_result_df['gpu'] = GPU_A100
RTX2080TI_result_df['gpu'] = GPU_RTX2080TI
RTX3050TI_result_df['gpu'] = GPU_RTX3050TI


all_result_df = pd.concat([A100_result_df, RTX2080TI_result_df, RTX3050TI_result_df])

all_result_df = all_result_df.sort_values(['gpu', 'Standard', 'File', 'chunksize_Bytes'])

print(all_result_df)


    Standard           File  chunks  decompression_throughput_GBps  \
8     snappy  500mb_dickens  131072                        19.8204   
9     snappy  500mb_dickens   32768                         7.5073   
10    snappy  500mb_dickens    8192                         2.0746   
11    snappy  500mb_dickens    2048                         0.5456   
12    snappy  500mb_dickens     512                         0.1682   
..       ...            ...     ...                            ...   
104     zstd      500mb_xml    2048                         0.4522   
105     zstd      500mb_xml     512                         0.2050   
106     zstd      500mb_xml     128                         0.0770   
107     zstd      500mb_xml      32                         0.0550   
108     zstd      500mb_xml       8                         0.0550   

     compression_throughput_GBps   ratio  chunksize_Bytes  \
8                         6.5454  1.3365           4000.0   
9                         2.7789  1.4

In [11]:
fig_gpus_througphut = px.line(
    all_result_df,
    title=f"Compression throughput", 
    x="chunksize_Bytes",
    y="compression_throughput_GBps",
    color="File",
    log_x=True,
    # log_y=True,
    markers=True,
    facet_row='Standard',
    facet_col="gpu",
    facet_col_spacing=0.05,
    width=1200,
)
# fig_gpus_througphut.update_xaxes(autorange="reversed")
# fig_gpus_througphut.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
fig_gpus_througphut.update_layout(
    yaxis_title="a",
    yaxis2_title="a",
    xaxis_title="Chunk size",
    xaxis2_title="Chunk size",
    xaxis3_title="Chunk size",
    # font_size=16,
)

fig_gpus_througphut.show()

In [12]:
fig_gpus_througphut = px.line(
    all_result_df,
    title=f"Decompression throughput", 
    x="chunksize_Bytes",
    y="decompression_throughput_GBps",
    color="File",
    log_x=True,
    markers=True,
    # log_y=True,
    facet_col='gpu',
    facet_row='Standard',
    facet_col_spacing=0.05,
    width=1200,
)
# fig_gpus_througphut.update_xaxes(autorange="reversed")
# fig_gpus_througphut.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
fig_gpus_througphut.update_layout(
    yaxis_title="Decompression throughput",
    xaxis_title="Chunk size",
    xaxis2_title="Chunk size",
    # font_size=16,
)

fig_gpus_througphut.show()

In [7]:
compressors = ['snappy', 'zstd']
compression_files = ['500mb_dickens', '500mb_mozilla', '500mb_mr', '500mb_nci', '500mb_samba', '500mb_sao', '500mb_silesia.zip', '500mb_xml', '500mb_lineitem.parquet']
approx_number_of_threads = [8, 32, 128, 512, 2048, 8192, 32768, 131072]


A100_utilisation_df = nsys_reader.NsysScanner(RTX3050TI_path, FILE_SIZE).get_utilisation_df(compression_files, compressors, approx_number_of_threads)
RTX2080TI_utilisation_df = nsys_reader.NsysScanner(RTX2080TI_path, FILE_SIZE).get_utilisation_df(compression_files, compressors, approx_number_of_threads)
RTX3050TI_utilisation_df = nsys_reader.NsysScanner(A100_path, FILE_SIZE).get_utilisation_df(compression_files, compressors, approx_number_of_threads)

A100_utilisation_df['gpu'] = GPU_A100
RTX2080TI_utilisation_df['gpu'] = GPU_RTX2080TI
RTX3050TI_utilisation_df['gpu'] = GPU_RTX3050TI


all_utilization_df = pd.concat([A100_utilisation_df, RTX2080TI_utilisation_df, RTX3050TI_utilisation_df])

all_utilization_df = all_utilization_df.sort_values(['gpu', 'standard', 'file', 'chunk_size'])
print(all_utilization_df)

../../../data/decompression/compare/RTX3050ti/output_snappy_500mb_dickens_8threads.h5
../../../data/decompression/compare/RTX3050ti/output_snappy_500mb_dickens_32threads.h5
../../../data/decompression/compare/RTX3050ti/output_snappy_500mb_dickens_128threads.h5
../../../data/decompression/compare/RTX3050ti/output_snappy_500mb_dickens_512threads.h5
../../../data/decompression/compare/RTX3050ti/output_snappy_500mb_dickens_2048threads.h5
../../../data/decompression/compare/RTX3050ti/output_snappy_500mb_dickens_8192threads.h5
../../../data/decompression/compare/RTX3050ti/output_snappy_500mb_dickens_32768threads.h5
../../../data/decompression/compare/RTX3050ti/output_snappy_500mb_dickens_131072threads.h5
../../../data/decompression/compare/RTX3050ti/output_zstd_500mb_dickens_8threads.h5
../../../data/decompression/compare/RTX3050ti/output_zstd_500mb_dickens_32threads.h5
../../../data/decompression/compare/RTX3050ti/output_zstd_500mb_dickens_128threads.h5
../../../data/decompression/compare/R

In [8]:
fig_gpus_utilization_compression = px.line(
    all_utilization_df,
    title=f"GPU utilization compression", 
    x="chunk_size",
    y="compression_utilization",
    color="file",
    log_x=True,
    markers=True,
    log_y=True,
    facet_col='gpu',
    facet_row='standard',
    facet_col_spacing=0.05,
    width=1200,
)
# fig_gpus_utilization_compression.update_xaxes(autorange="reversed")
# fig_gpus_utilization_compression.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
# fig_gpus_utilization_compression.update_layout(
#     yaxis_title="Compression ratio",
#     xaxis_title="Chunk size",
#     xaxis2_title="Chunk size",
#     font_size=16,
# )
print(all_utilization_df.to_string())
fig_gpus_utilization_compression.show()

    standard  chunk_size                    file  compression_utilization  decompression_utilization        gpu
7     snappy        4000           500mb_dickens                35.936501                  57.991657       A100
6     snappy       16000           500mb_dickens                39.227277                  59.927612       A100
5     snappy       64000           500mb_dickens                67.099357                  66.812108       A100
4     snappy      256000           500mb_dickens                90.332419                  89.850706       A100
3     snappy     1024000           500mb_dickens                46.605675                  47.321909       A100
2     snappy     4096000           500mb_dickens                11.913052                  12.044099       A100
1     snappy    16384000           500mb_dickens                 4.998948                   4.999741       A100
0     snappy    65536000           500mb_dickens                 4.999148                   4.999619    

In [9]:
fig_gpus_utilization_decompression = px.line(
    all_utilization_df,
    title=f"GPU utilization decompression", 
    x="chunk_size",
    y="decompression_utilization",
    color="file",
    log_x=True,
    markers=True,
    log_y=True,
    facet_col='gpu',
    facet_row='standard',
    facet_col_spacing=0.05,
    width=1200,
)
# fig_gpus_utilization_decompression.update_xaxes(autorange="reversed")
# fig_gpus_utilization_decompression.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
# fig_gpus_utilization_decompression.update_layout(
#     yaxis_title="Compression ratio",
#     xaxis_title="Chunk size",
#     xaxis2_title="Chunk size",
#     font_size=16,
# )

fig_gpus_utilization_decompression.show()